In [ ]:
from PIL import Image
import torch
from torch.utils import data
import numpy as np
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm


In [ ]:
train_transformer=transforms.Compose(
[  
   transforms.RandomHorizontalFlip(0.2),
   transforms.RandomRotation(68),
    transforms.RandomGrayscale(0.2),
   transforms.Resize((299,299)),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.5,0.5,0.5],
                std=[0.5,0.5,0.5]) 
]
)
test_transformer=transforms.Compose(
[ 
   transforms.Resize((299,299)),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.5,0.5,0.5],
                std=[0.5,0.5,0.5]) 
]
)


In [ ]:
train_dataset=torchvision.datasets.ImageFolder(
  'E:/blood-cells/dataset2-master/dataset2-master/images/TRAIN',
    transform=train_transformer
)

test_dataset=torchvision.datasets.ImageFolder(
  'E:/blood-cells/dataset2-master/dataset2-master/images/TEST',
   transform=test_transformer
)
  

In [ ]:
train_dataset.classes

In [ ]:
train_dataset.class_to_idx

In [ ]:
id_to_class={}
for k,v in train_dataset.class_to_idx.items():
    print(k,v)
    id_to_class[v]=k
id_to_class

In [ ]:
Batch_size=12
dl_train=torch.utils.data.DataLoader(
        train_dataset,
        batch_size=Batch_size,
        shuffle=True
)
dl_test=torch.utils.data.DataLoader(
        test_dataset,
        batch_size=Batch_size,
        shuffle=True
)

In [ ]:
img,label=next(iter(dl_train))

In [ ]:
img.shape

In [ ]:
im=img[0]

In [ ]:
im.shape

In [ ]:
label[0]

In [ ]:
im.shape

In [ ]:
im=im.permute(1,2,0)

In [ ]:
im-im.numpy()
im=(im+1)/2
plt.imshow(im)

In [ ]:
plt.figure(figsize=(12,8))
for i,(img,label) in enumerate(zip(img[:8],label[:8])):
    img=(img.permute(1,2,0).numpy()+1)/2
    plt.subplot(2,4,i+1)
    plt.title(id_to_class.get(label.item()))
    plt.imshow(img)    

In [ ]:
##################建立神经网络

In [ ]:
model=torchvision.models.Inception-Resnet-v2(init_weights=True)

model.fc.out_features=4
model

In [ ]:
img,label=next(iter(dl_train))
img.shape


In [ ]:
model.eval()
pred=model(img)
pred


In [ ]:
loss_fn=nn.CrossEntropyLoss()

In [ ]:

optim=torch.optim.Adam(model.parameters(),lr=0.00002)


In [ ]:
if torch.cuda.is_available():
    model.to('cuda')
torch.cuda.is_available()

In [ ]:
def fit(epoch, model, trainloader, testloader):
    correct = 0
    total = 0
    running_loss = 0
    model.train()
    for x, y in tqdm(trainloader):
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x).logits
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
        
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
        
        
    test_correct = 0
    test_total = 0
    test_running_loss = 0 
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(testloader):
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total
    
        
    print('epoch: ', epoch, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'test_loss： ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
             )
        
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc

In [ ]:
epochs = 300

In [ ]:
# train_loss = []
# train_acc = []
# test_loss = []
# test_acc = []

# for epoch in range(epochs):
#     epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch,
#                                                                  model,
#                                                                  dl_train,
#                                                                  dl_test)
#     train_loss.append(epoch_loss)
#     train_acc.append(epoch_acc)
#     test_loss.append(epoch_test_loss)
#     test_acc.append(epoch_test_acc)

In [ ]:
plt.plot(range(1, epochs+1), train_loss, label='train_loss')
plt.plot(range(1, epochs+1), test_loss, label='test_loss')
plt.legend()

In [ ]:
plt.plot(range(1, epochs+1), train_acc, label='train_acc')
plt.plot(range(1, epochs+1), test_acc, label='test_acc')
plt.legend()

In [ ]:
torch.save(model,'Bloodcell.pth')

In [ ]:
model=torch.load('Bloodcell.pth')
model

In [ ]:
img,label=next(iter(dl_test))
img.shape

In [ ]:
img=img.to('cuda')
pred=model(img)
pred.shape

In [ ]:
pred_re=torch.argmax(pred, dim=1)
pred_re
pred_re=pred_re.cpu().numpy()
pred_re=pred_re.tolist()
pred_re

In [ ]:

for i in pred_re[0:8]:
    print(id_to_class[i])
id_to_class[pred_re[0:8][1]]

In [ ]:
plt.figure(figsize=(16,8))
img=img.cpu()
for i,(img,label) in enumerate(zip(img[:8],label[:8])):
    img=(img.permute(1,2,0).numpy()+1)/2
    plt.subplot(2,4,i+1)
    pred_title=id_to_class[pred_re[0:8][i]]
    plt.title('R:{},P:{}'.format(id_to_class.get(label.item()),pred_title))
    plt.imshow(img) 